In [1]:
import pyspark

In [4]:
!pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import sys 
sys.path.append("/opt/workspace")
from load_env import env_parameter

# Load from .env
env_vars = env_parameter()

In [9]:
jdbc_url = env_vars["jdbc_url"]
db_user = env_vars["db_user"]
db_password = env_vars["db_password"]
spark_master = env_vars["spark_master"]

In [10]:
from pyspark.sql import SparkSession

# Start Spark session with PostgreSQL JDBC driver
spark = SparkSession.builder \
    .appName("ReadCRM") \
    .master(spark_master) \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.5.0") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "file:/opt/spark-events") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-75bc2518-3167-47f4-a6da-90f50876389b;1.0
	confs: [default]
	found org.postgresql#postgresql;42.5.0 in central
	found org.checkerframework#checker-qual;3.5.0 in central
downloading https://repo1.maven.org/maven2/org/postgresql/postgresql/42.5.0/postgresql-42.5.0.jar ...
	[SUCCESSFUL ] org.postgresql#postgresql;42.5.0!postgresql.jar (1534ms)
downloading https://repo1.maven.org/maven2/org/checkerframework/checker-qual/3.5.0/checker-qual-3.5.0.jar ...
	[SUCCESSFUL ] org.checkerframework#checker-qual;3.5.0!checker-qual.jar (454ms)
:: resolution report :: resolve 2001ms :: artifacts dl 1995ms
	:: modules in use:
	org.checkerframework#checker-qual;3.5.0 from central in [default]
	org.postgresql#postgresql;42.5.0 from central in [default]
	----------------------------------------------

In [11]:
jdbc_properties = {
    "user": db_user,
    "password": db_password,
    "driver": "org.postgresql.Driver"
}

In [12]:
df = spark.read.jdbc(url=jdbc_url, table="customers", properties=jdbc_properties)
df.show()

+---+-----------------+-------------+-------------+--------+--------------------+-------------+--------------------+--------------------+
| id|             name|     industry|       region|   phone|             website|         tags|            metadata|          created_at|
+---+-----------------+-------------+-------------+--------+--------------------+-------------+--------------------+--------------------+
|  1|Northwind Traders|       Retail|North America|555-1001|https://northwind...|[active, key]|{"customer_tier":...|2025-06-02 17:19:...|
|  2|      Contoso Ltd|   Technology|       Europe|555-1002| https://contoso.com|   [prospect]|{"platform": "Azu...|2025-06-02 17:19:...|
|  3|  Adventure Works|Manufacturing|         Asia|555-1003|https://adventure...|        [new]|{"needs": "IoT in...|2025-06-02 17:19:...|
+---+-----------------+-------------+-------------+--------+--------------------+-------------+--------------------+--------------------+



In [13]:
spark.stop()